# Data Processing (통계적 분석)
- 상관관계가 큰 변수 찾기
- 결측치 처리
- 이상치 처리
- 로그 변환
- 데이터 (주택가격 예측 데이터)
- https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## data analysis and cleaning
- correlation
- missing values
- using house saleprice estimation

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.shape

### analyze target variable
- statistic analysis
- distribution and histogram

In [ ]:
df.describe()

In [ ]:
df.describe().shape

In [ ]:
plt.hist(df['SalePrice'], bins=100)

In [ ]:
sns.distplot(df['SalePrice'], kde=True); # kde - kernel density estimation

In [ ]:
sns.distplot(np.log1p(df['SalePrice']), kde=True)  # log-normal dist

In [ ]:
df.corr()

In [ ]:
df.corr().shape

### objctive analysis
- correlation (heatmap and matrix)
- Scatter plots between the most correlated variables

In [ ]:
corrmat = df.corr()

f, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
corrmat.sort_values(by='SalePrice', ascending=False).T.tail(5)

In [ ]:
k=7
cols = corrmat.nlargest(k, 'SalePrice')
cols

- scatter graph (target variable and numerical data feature)

In [ ]:
var1 = 'GrLivArea'
var2 = 'GarageArea'
var3 = 'TotalBsmtSF'

plt.figure(figsize=(12,8))
plt.subplot(1, 3, 1)
plt.ylim(0, 800000)
plt.scatter(df[var1], df.SalePrice)
plt.subplot(1, 3, 2)
plt.scatter(df[var2], df.SalePrice)
plt.subplot(1, 3, 3)
plt.scatter(df[var3], df.SalePrice)

- target variable and categorical feature (boxplot is better)

In [ ]:
print(df.shape)
print(df['OverallQual'].unique())
print(df['OverallQual'].dtypes)
df['OverallQual'].value_counts()

In [ ]:
plt.scatter(df.OverallQual, df.SalePrice);

In [ ]:
plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=df)
fig.axis(ymin=0, ymax=800000);

In [ ]:
df['GarageCars'].unique()

In [ ]:
plt.figure(figsize=(10,12))
plt.subplot(2,1,1)
plt.title('Bivariable analysis for categorical data')
sns.boxplot(data=df, x='GarageCars', y='SalePrice')
plt.subplot(2,1,2)
sns.boxplot(data=df, x='YearBuilt', y='SalePrice')
plt.xticks(rotation=90)

- we found that 
  - positive correlation between target and 'OverallQual'
  - not strictly positive (weak) correlation between target and 'GarageCars' and 'YearBuilt'

In [ ]:
cols = cols['SalePrice'].index
cols

In [ ]:
cm = df[cols].corr()

In [ ]:
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True,
        fmt='.2f', annot_kws={'size': 10}, 
        yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 
        'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df[cols], size = 2.5)
plt.show();

### Missing Values

In [ ]:
df.describe()

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
df.count().sort_values()

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
# or
df.count().sort_values()   # count non-NA cells

In [ ]:
total = df.isna().sum().sort_values(ascending=False)
percent = (df.isna().sum()/df.shape[0]).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
garages = ['GarageCond','GarageType','GarageYrBlt','GarageFinish','GarageQual', 'GarageCars', 'GarageArea']
bsmts = ['BsmtExposure','BsmtFinType2','BsmtFinType1','BsmtCond','BsmtQual']
print(df[garages].describe())
df[garages].info()

In [ ]:
df[garages].describe(include=['O'])   # 'object'

In [ ]:
df[garages].describe(include='all')

In [ ]:
# object(string) variables
print(df['GarageCond'].unique())
print(df['GarageType'].unique())
print(df['GarageFinish'].unique())
print(df['GarageQual'].unique())

In [ ]:
df[garages].select_dtypes(include=['object'])

In [ ]:
df2 = df.copy()
df2['GarageCond'].unique()

In [ ]:
df2 = pd.concat([df[garages], df['SalePrice']], axis=1)

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df2 = df2.dropna()
df2.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
df2['GarageCond'] = LabelEncoder().fit_transform(df2['GarageCond'])
df2['GarageType'] = LabelEncoder().fit_transform(df2['GarageType'])
df2['GarageFinish'] = LabelEncoder().fit_transform(df2['GarageFinish'])
df2['GarageQual'] = LabelEncoder().fit_transform(df2['GarageQual'])

In [ ]:
df2.dtypes

In [ ]:
sns.pairplot(df2, size = 2.5)

In [ ]:
df[bsmts].select_dtypes(include=['object'])

In [ ]:
df2 = df.copy()
df2 = pd.concat([df[bsmts], df['SalePrice']], axis=1)
df2 = df2.dropna()
df2['BsmtExposure'] = LabelEncoder().fit_transform(df2['BsmtExposure'])
df2['BsmtFinType2'] = LabelEncoder().fit_transform(df2['BsmtFinType2'])
df2['BsmtFinType1'] = LabelEncoder().fit_transform(df2['BsmtFinType1'])
df2['BsmtCond'] = LabelEncoder().fit_transform(df2['BsmtCond'])
df2['BsmtQual'] = LabelEncoder().fit_transform(df2['BsmtQual'])
sns.pairplot(df2, height = 2.5)

In [ ]:
df2 = df[['MasVnrArea', 'MasVnrType', 'OverallQual', 'SalePrice']]
df2 = df2.dropna()
df2['MasVnrType'] = LabelEncoder().fit_transform(df2['MasVnrType'])
sns.pairplot(df2, height = 2.5)

###  make strategy (cleaning)
- missing rate greater than 15% - remove features
- keep Garages\* and Bsmt\*
- keep MasVnr\*.

In [ ]:
new_df = df.copy()
new_df = new_df.drop((missing_data[missing_data['Total'] > 100]).index, axis=1)
new_df = new_df.dropna()
print(new_df.shape, df.shape)
print(new_df.isna().sum().max())

### Outliers
- bivariate analysis

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
saleprice_scaled = sc.fit_transform(np.array(new_df['SalePrice']).reshape(-1,1))
saleprice_scaled

In [ ]:
saleprice_scaled[:, 0]

In [ ]:
saleprice_scaled[saleprice_scaled[:,0].argsort()]

In [ ]:
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
low_range, high_range

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(new_df['SalePrice'])  #  if not shown, x-axis is default len(y)
plt.subplot(1,2,2)
plt.plot(saleprice_scaled)
plt.show()

In [ ]:
var1 = 'GrLivArea'    # most significant nominal features
var2 = 'TotalBsmtSF'
var3 = 'OverallQual'
plt.figure(figsize=(12,6))
plt.subplot(1, 3, 1)
plt.scatter(new_df[var1], new_df['SalePrice'])
plt.subplot(1, 3, 2)
plt.scatter(new_df[var2], new_df['SalePrice'])
plt.subplot(1, 3, 3)
plt.scatter(new_df[var3], new_df['SalePrice'])


- top two points: greater than 7, but keep the trend
- rightmost two points:  remove

In [ ]:
# deleting points
new_df.sort_values(by='GrLivArea', ascending=False)[:2]

In [ ]:
new_df.sort_values(by='TotalBsmtSF', ascending=False)[:5]

In [ ]:
new_df = new_df.drop(new_df[new_df['Id'] == 1299].index)
new_df = new_df.drop(new_df[new_df['Id'] == 524].index)
new_df.shape

### Compare before and after preprocessing
- use 5 most significant features
- use all features (need categorical encoding)



In [ ]:
df.shape, new_df.shape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
np.random.seed(42)
corrmat = df.corr()
cols = corrmat.nlargest(5, 'SalePrice')
print(cols.index)
df3 = df[cols.index]
X, y = df3.values[:, 1:], df3.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("Before the data cleaning: ", model.score(X_test, y_test))

In [ ]:
np.random.seed(42)
corrmat = new_df.corr()
cols = corrmat.nlargest(5, 'SalePrice')
print(cols.index)
df3 = new_df[cols.index]
X, y = df3.values[:, 1:], df3.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("After the data cleaning: ", model.score(X_test, y_test))

In [ ]:
# use all features

df.shape, new_df.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
new_df.dtypes

In [ ]:
df3 = new_df.copy()
for i in range(1, len(df3.dtypes)-1):
    if df3.dtypes[i] == 'object':
        df3.iloc[:,i] = LabelEncoder().fit_transform(df3.iloc[:,i])

In [ ]:
(df3.dtypes == 'object').sum()

In [ ]:
df3.columns[-1]

In [ ]:
np.random.seed(42)
X, y = df3.values[:, 1:-1], df3.values[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("Using all the features: ", model.score(X_test, y_test))

In [ ]:
df4 = new_df.copy()
obj_cols = df4.select_dtypes(include=['object']).columns

In [ ]:
obj_cols

In [ ]:
df4.shape

In [ ]:
df4 = pd.get_dummies(df4, columns=obj_cols)
df4.shape

In [ ]:
df4.columns[-1]

In [ ]:
df4['SalePrice'].values

In [ ]:
np.random.seed(17)
X, y = df4.drop(['Id','SalePrice'], axis=1).values, df4['SalePrice'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("Using all the features (onehot encoding): ", model.score(X_test, y_test))